# SVD Predictor

Creating and testing the SVD Predictor model on our dataframe. First we need to split the data in train and test set, and then run GridSearchCV on the training set in order to find the best n factors value to pass into the SVD model creation. 

In [29]:
import pickle
from datetime import datetime
from tqdm import tqdm
import numpy as np
import pandas as pd
import os
import sklearn
from sklearn.metrics.pairwise import cosine_similarity
import random
from surprise import Reader, Dataset
from surprise import SVD
from surprise import SVDpp
from surprise.model_selection import GridSearchCV
import xgboost as xbg
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [27]:
movie = pd.read_pickle("cleanedMovie.pkl")
movie.head()

,MovieID,CustomerID,Rating
0,1,1488844,3
3,1,30878,4
7,1,1248029,3
19,1,372233,5
20,1,1080361,3


In [32]:
movie_train, movie_test = sklearn.model_selection.train_test_split(movie, train_size = 0.8)

(3751486, 3)

In [34]:
print(movie_test.shape)
movie_test.head()

(3751486, 3)


,MovieID,CustomerID,Rating
7882419,1582,826184,1
99705020,17580,1833576,3
45681607,8171,395747,5
22273012,4225,2361318,4
55260630,10109,864647,5


In [36]:
print(movie_train.shape)
movie_train.head()

(15005940, 3)


,MovieID,CustomerID,Rating
11836212,2284,1352912,4
88353372,15717,574834,4
3977489,758,698147,4
76706859,13923,1200886,3
85305777,15134,2222815,3


In [37]:
reader = Reader(rating_scale=(1,5))
movieInput = pd.DataFrame()
movieInput['CustomerID'] = movie_train['CustomerID']
movieInput['MovieID'] = movie_train['MovieID']
movieInput['Rating'] = movie_train['Rating']

train_data = Dataset.load_from_df(movieInput, reader)
trainset = train_data.build_full_trainset()

In [38]:
testset = list(zip(movie_test["CustomerID"].values, movie_test["MovieID"].values, movie_test["Rating"].values))

In [40]:
error_table = pd.DataFrame(columns = ["Model", "Train_RMSE", "Test_RMSE"])

In [46]:
trainset.global_mean

3.4188866542182628

## Creating Utility Functions

In [59]:
def make_table(model_name, rmse_train, rmse_test):
    global error_table
    error_table = error_table.append(pd.DataFrame([[model_name, rmse_train, rmse_test]], columns = ["Model", "Train_RMSE", "Test_RMSE"]))
    error_table.reset_index(drop = True, inplace = True)

In [60]:
def run_surprise(algo, trainset, testset, model_name):
    start = datetime.now()
    algo.fit(trainset)
    
    pred_train = algo.test(trainset.build_testset())
    
    trainActual = np.array([p.r_ui for p in pred_train])
    trainPred = np.array([p.est for p in pred_train]) 
    trainRMSE = np.sqrt(mean_squared_error(trainActual, trainPred))
    
    print("Train Data RMSE: {}".format(trainRMSE))
    print("\n")
    
    train = {"RMSE": trainRMSE, "Prediction": trainPred}
    
    pred_test = algo.test(testset)
    testActual = np.array([p.r_ui for p in pred_test])
    testPred = np.array([p.est for p in pred_test])
    testRMSE = np.sqrt(mean_squared_error(testActual, testPred))
    
    print("Test Data RMSE: {}".format(testRMSE))
    print("\n")
    
    test = {"RMSE": testRMSE, "Prediction": testPred}
    
    print("Time Taken = " + str(datetime.now() - start))
    
    make_table(model_name, trainRMSE, testRMSE)
    
    return train, test

## Finding N Factors

GridSearchCV cannot handle the amount of data we are passing through, so we will run GridSearchCV on a smaller portion of the dataset in order to return the best n_factors to pass into SVD. SVD will itself get the full dataset we are using.

In [66]:
params = { 'n_factors': [5, 10, 15, 20, 25, 30, 35, 40, 50]}
grid = GridSearchCV(SVD, params, measures=['rmse'], cv=3, refit=True)
grid.fit(Dataset.load_from_df(movieInput.iloc[:1500000], reader))
print(grid.best_score['rmse'])

0.9141425899963144


Of the N factors passed in, we can find the one that had the best RMSE and use that in the SVD model. Below, we use that directly from the calculation above. In the following class file, we use the value as a static variable in order to minimize processing time on unnecessary calculations.

In [67]:
algo = SVD(n_factors = grid.best_params['rmse']['n_factors'], biased=True, verbose=True)
train_result, test_result = run_surprise(algo, trainset, testset, "SVD")

Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 10
Processing epoch 11
Processing epoch 12
Processing epoch 13
Processing epoch 14
Processing epoch 15
Processing epoch 16
Processing epoch 17
Processing epoch 18
Processing epoch 19
Train Data RMSE: 0.8198083205483772


Test Data RMSE: 0.828587192328288


Time Taken = 0:10:29.761314


In [136]:
algo.predict(6, 16377)

Prediction(uid=6, iid=16377, r_ui=None, est=4.215468759306557, details={'was_impossible': False})

## Putting it all together

Take all the information we gathered, the functions we built, and the models we created, and put them all into one class. The class saves algorithms as pickle files to be reused later without having to calculate the model and algorithm all over again. The class also has no testset, as there is no need for verification at this stage- only fitting the model and predicting values for the given user.

In [113]:
import pickle
from tqdm import tqdm
import numpy as np
import pandas as pd
import os
import pathlib
from surprise import Reader, Dataset
from surprise import SVD
from surprise import SVDpp
from surprise.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

class SVDPredictor:
    error_table = pd.DataFrame(columns = ["Model", "Train_RMSE", "Test_RMSE"])
    
    #Class takes in final.csv as a whole as a DataFrame
    def __init__(self, data):
        self.movie = data
        self.createAlgorithmFromData()
        
    def createAlgorithmFromData(self):
        #check if algo and trainset/train_data files are already created
        file = pathlib.Path('svd.pickle')
        if not file.exists():
            self._reduceDataSize()
            self._splitMovie()
            self._createTrainSet()
        self._run_surprise()
        
    def predict(self, userID, movieID):
        #use algo to predict rating. Return predicted rating
        return self.algo.predict(userID, movieID)
    
    def _splitMovie(self):
        self.movie = self.movie.iloc[:1500000]
        
    def _createTrainSet(self):
        reader = Reader(rating_scale=(1,5))
        movieInput = pd.DataFrame()
        movieInput['CustomerID'] = self.movie['CustomerID']
        movieInput['MovieID'] = self.movie['MovieID']
        movieInput['Rating'] = self.movie['Rating']

        self.train_data = Dataset.load_from_df(movieInput, reader)
        self.trainset = self.train_data.build_full_trainset()
        #write to a file
    
    def _reduceDataSize(self):
        self.movie['Date'] = self.movie['Date'].astype('category')
        self.movie['MovieID'] = self.movie['MovieID'].astype('int16')
        self.movie['CustomerID'] = self.movie['CustomerID'].astype('int32')
        self.movie['Rating'] = self.movie['Rating'].astype('int8')
    
    def _run_surprise(self): 
        file = pathlib.Path('svd.pickle')
        if file.exists():
            with open('svd.pickle', 'rb') as f:
                self.algo = pickle.load(f)
        else:
            self.algo = SVD(n_factors = 5, biased=True, verbose=True)
            self.algo = self.algo.fit(self.trainset)
            with open('svd.pickle', 'wb') as f:
                pickle.dump(self.algo, f)

    def _make_table(model_name, rmse_train, rmse_test):
        global error_table
        error_table = error_table.append(pd.DataFrame([[model_name, rmse_train, rmse_test]], columns = ["Model", "Train_RMSE", "Test_RMSE"]))
        error_table.reset_index(drop = True, inplace = True)

In [114]:
svd = SVDPredictor(movie)
print(svd.predict(1, 5).est)

3.9187944579396867


In [145]:
print(svd.predict(3, 16378).est)

3.6377646666666665
